In [1]:
# !pip install --upgrade numpy scipy scikit-learn --user
# !pip show numpy scipy scikit-learn

In [2]:
import  pandas as pd
import numpy as np
df = pd.read_csv('merged_df_abstract_preprocess_all.csv')
df = df.dropna(subset=['processed_abstract'])

/var/folders/rg/q91_28dx1cs9z4ty_4g_hrwc0000gn/T/ipykernel_13464/217804304.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('merged_df_abstract_preprocess_all.csv')


In [3]:
print(df.shape)
print(df.columns)

(311557, 20)
Index(['Title', 'Abstract', 'Disciplines', 'Keywords', 'Source',
       'Disciplines_split', 'Astronomy', 'Biology', 'Chemistry',
       'Computer Science', 'Economics', 'Education & instruction',
       'Electrical Engineering', 'European & international law', 'History',
       'Mathematics', 'Physics', 'Social Sciences', 'Statistics',
       'processed_abstract'],
      dtype='object')


In [4]:
# Preprocessing
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Vectorize the Text Data using TF-IDF
vectorizer_Logistic_one_hot = TfidfVectorizer()
X = vectorizer_Logistic_one_hot.fit_transform(df['processed_abstract'])  # Text data turned into numerical format

# Define the labels (One-hot encoded)
labels = [
    'Astronomy', 'Biology', 'Chemistry', 'Computer Science', 'Economics', 'Education & instruction',
    'Electrical Engineering', 'European & international law', 'History', 'Mathematics', 'Physics',
    'Social Sciences', 'Statistics'
]

y = df[labels]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Save vectorizer
import joblib
joblib.dump(vectorizer_Logistic_one_hot, '1_vectorizer_Logistic_one_hot.joblib')
vectorizer_Logistic_one_hot = joblib.load('1_vectorizer_Logistic_one_hot.joblib')

In [6]:
# 1 Logistic
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier

logreg = LogisticRegression(max_iter=1000)
Model_Logistic_Multi = MultiOutputClassifier(logreg)
Model_Logistic_Multi.fit(X_train, y_train)

y_pred = Model_Logistic_Multi.predict(X_test)

report_Logistic_Multi = classification_report(y_test, y_pred, target_names=labels)
print(report_Logistic_Multi)

                              precision    recall  f1-score   support

                   Astronomy       0.98      0.92      0.95      4527
                     Biology       0.93      0.41      0.57       292
                   Chemistry       0.73      0.13      0.22       291
            Computer Science       0.94      0.93      0.94     28514
                   Economics       0.73      0.25      0.37       561
     Education & instruction       0.79      0.41      0.54       182
      Electrical Engineering       0.76      0.50      0.60      4363
European & international law       0.93      0.26      0.40        97
                     History       0.85      0.29      0.43       134
                 Mathematics       0.92      0.88      0.90     12638
                     Physics       0.94      0.92      0.93     17342
             Social Sciences       0.92      0.30      0.45       216
                  Statistics       0.84      0.53      0.65      1662

                  

/Users/claraduchossois/Documents/MADS/MADS Sem 3/NLP/env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [7]:
# Save model
import joblib
joblib.dump(Model_Logistic_Multi, 'model/1_Model_Logistic_Multi.joblib')
Model_Logistic_Multi = joblib.load('model/1_Model_Logistic_Multi.joblib')

FileNotFoundError: [Errno 2] No such file or directory: 'model/1_Model_Logistic_Multi.joblib'

In [ ]:
import numpy as np

feature_names = np.array(vectorizer_Logistic_one_hot.get_feature_names_out())
coefficients = Model_Logistic_Multi.estimators_

top_n = 10

for i, label in enumerate(labels):
    coef = coefficients[i].coef_[0]
    top_keywords_idx = np.argsort(coef)[-top_n:][::-1]
    print(label)

    for idx in top_keywords_idx:
        print(f"{feature_names[idx]}: {coef[idx]}")
    print()


Astronomy
telescope: 10.61206887020964
galaxy: 10.542583954989995
star: 8.44685975945685
planet: 8.22455796472463
observation: 7.7490029506349485
stellar: 7.163741995434713
redshift: 7.097421063272092
sky: 6.77311948449089
solar: 6.339692439438203
atmosphere: 6.337198250973576

Biology
protein: 6.65425505777634
receptor: 5.814041213731296
neuronal: 5.5583858255848275
activity: 5.542227377288786
neuron: 5.2599464392938975
gene: 5.166189910678739
cell: 4.959676495028663
neural: 4.796108374311479
stimulus: 4.602840148150857
brain: 4.230271826926343

Chemistry
molecule: 7.21337000435448
molecular: 6.398952650157769
reaction: 5.645484579603119
energy: 5.3645313617222214
electron: 4.43659695261979
electronic: 4.100129851294552
chemical: 4.079821378273239
calculation: 3.7382054447463706
excited: 3.404122406470716
dynamic: 3.289124323409524

Computer Science
robot: 11.032094955786354
language: 10.318395216834526
dataset: 9.44480251837512
task: 7.753036299389881
art: 7.089228669658545
code: 6.7

In [ ]:
# Creating the 'preprocess' function
import spacy
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from textblob import TextBlob
import string

# Load spaCy model
nlp = spacy.load('en_core_web_sm')

def preprocess(text):
    text_sub = re.sub(r"^\[en\]\s*", "", text) # delete the [en] in the beginning
    # text_rm_punc = text_sub.translate(str.maketrans("", "", string.punctuation)) # Remove punctuation in the following
    doc = nlp(str(text_sub))

    # Tokenization and Remove unwanted characters
    tokens = [token.text for token in doc if not token.is_punct]  # Removing punctuation and get tokens
    # print("Cleaned Tokens:", tokens)

    # Stop Word Removal
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words and token.strip()]
    # print("Filtered Tokens:", filtered_tokens)

    # Lemmatization
    lemmatized_tokens = [token.lemma_ for token in doc if token.text.lower() in filtered_tokens]
    # print("Lemmatized Tokens:", lemmatized_tokens)

    # Stemming
    # stemmer = PorterStemmer()
    # stemmed_tokens = [stemmer.stem(token) for token in lemmatized_tokens] # Didn't do this step to remain the meaning of the words
    # print("Stemmed Tokens:", stemmed_tokens)

    # Correct spelling using TextBlob
    # corrected_text = ' '.join(stemmed_tokens)  # Join tokens into a string
    # blob = TextBlob(corrected_text)
    # corrected_review = blob.correct()
    # print("Corrected Review:", corrected_review)

    return ' '.join(lemmatized_tokens) # Return lemmatized tokens as a space-separated string


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "D:\软件\Python\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "D:\软件\Python\Lib\site-packages\traitlets\config\application.py", line 1077, in launch_instance
    app.start()
  File "D:\软件\Python\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "D:\软件\Python\Lib\site-packages\tornado\platform\asyncio.py", line 205, in 

In [ ]:
new_text = "This is a new scientific paper on artificial intelligence in physics and astronomy."
new_text_processed = preprocess(new_text)
new_text_processed_vectorizer = vectorizer_Logistic_one_hot.transform([new_text_processed])  # vectorizer in training


predicted_probabilities = Model_Logistic_Multi.predict_proba(new_text_processed_vectorizer)

print(f"new text: {new_text}")
print(f"new text processed: {new_text_processed}")


print("Predicted probabilities structure:", predicted_probabilities)


predicted_labels = []


for i, label in enumerate(labels):
    prob = predicted_probabilities[i][0][1]

    if prob > 0.5:
        predicted_labels.append(label)
print(f"Predicted labels: {predicted_labels}")


new text: This is a new scientific paper on artificial intelligence in physics and astronomy.
new text processed: new scientific paper artificial intelligence physics astronomy
vectorizer: [[0. 0. 0. ... 0. 0. 0.]]
Predicted probabilities structure: [array([[0.6979679, 0.3020321]]), array([[0.995078, 0.004922]]), array([[0.99881607, 0.00118393]]), array([[0.91967136, 0.08032864]]), array([[0.98833598, 0.01166402]]), array([[0.99792144, 0.00207856]]), array([[0.98454621, 0.01545379]]), array([[0.99734856, 0.00265144]]), array([[0.9955249, 0.0044751]]), array([[0.63706154, 0.36293846]]), array([[0.90728666, 0.09271334]]), array([[0.99646474, 0.00353526]]), array([[0.99519003, 0.00480997]])]
Predicted labels: []
